In [ ]:
from google.colab import auth
from datetime import datetime
auth.authenticate_user()
!gsutil -m cp -r gs://predict_cfs/meta2 .
!gsutil -m cp -r  gs://predict_cfs/meta3 .
!gsutil -m cp -r  gs://predict_cfs/meta4 .

In [ ]:
import torch
class MetaDataEncoder(torch.nn.Module):
  def __init__(self, number_meta_items, end_dimension, n_layers):
    super().__init__()
    self.number_meta_items = number_meta_items
    self.end_dimension = end_dimension
    self.linear_layer_array = []
    for i in range(0, n_layers-1):
      self.linear_layer_array.append(torch.nn.Linear(number_meta_items, number_meta_items))
    self.final_layer = torch.nn.Linear(number_meta_items, end_dimension)
    self.max_flow = torch.nn.Linear(end_dimension, 1)
    self.min_flow = torch.nn.Linear(end_dimension, 1)
  def forward(self, x):
    for layer in self.linear_layer_array:
      x = layer(x)
    x = self.final_layer(x)
    return self.max_flow(x), self.min_flow(x)



In [ ]:
c = MetaDataEncoder(2, 128, 10)

In [ ]:
dummy_data = torch.rand(1, 6)
c(dummy_data)

(tensor([[-0.2160]], grad_fn=<AddmmBackward>),
 tensor([[-0.1116]], grad_fn=<AddmmBackward>))

In [ ]:
from torch.utils.data import Dataset
from typing import List
import os
import json
class DataLoading(Dataset):
  def __init__(self, data_directory: str, meta_keys:List[str], target_keys:List[str]):
    self.file_list = sorted(os.listdir(data_directory))
    self.dir_path = data_directory
    self.key_list = meta_keys
    self.target_keys = target_keys

  def __getitem__(self, idx):
    file_name = self.file_list[idx]
    file_path = os.path.join(self.dir_path, file_name)
    source_data = []
    targs = {}
    with open(file_path) as f:
      data = json.load(f)
    for key, value in data.items():
      if key in self.key_list:
        source_data.append(value)
      elif key in self.target_keys:
        targs[key] = value
    return torch.FloatTensor(source_data), targs




In [ ]:
test_loader = DataLoading("meta2", ["nan_flow", "nan_precip"], ["max_flow", "min_flow"])

In [ ]:
test_loader[0]

(tensor([12931.,   729.]), {'max_flow': 41300.0, 'min_flow': 91.3})

In [ ]:
from torch import optim
criterion = torch.nn.MSELoss()
opt = optim.Adam(c.parameters(), lr=0.0001)
for epoch in range(10):
  i = 0
  running_loss = 0.0
  for src, trg in test_loader:
    opt.zero_grad()
    # TODO figure how to avoid
    output = c(src)
    min_loss = criterion(output[0], torch.as_tensor(trg["max_flow"]))
    max_loss = criterion(output[1], torch.as_tensor(trg["min_flow"]))
    min_loss.backward()
    max_loss.backward()
    opt.step()
    running_loss += min_loss.item() + max_loss.item()
    i+=1

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored